In [17]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.tools import tool
from langchain import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent
from typing import Optional
import requests
from langchain.tools.base import StructuredTool
from dotenv import load_dotenv
import os
import time
# Load environment variables from .env file
load_dotenv()

True

In [20]:
@tool
def post_message(url: str, body: dict, parameters: Optional[dict] = None) -> str:
    """Sends a POST request to the given url with the given body and parameters."""
    result = requests.post(url, json=body, params=parameters)
    return f"Status: {result.status_code} - {result.text}"

In [13]:
import requests
from langchain.tools import BaseTool
from typing import List

class IPFSUploadTool(BaseTool):
    
    name = "IPFSUploadTool"
    description = """Uploads files to IPFS using web3stroage.
        The only paramater is paths, which is a list of paths such as the following. It will accept a list of paths in json format for files.
will be returned is a json body"""

    def _run(self, paths: str) -> str:
        """Uploads files to IPFS using web3stroage.
        The only paramater is paths, which is a list of paths such as the following. It will accept a list of paths in json format for files.
        ["paths/blob.json"]
        What will be returned is a json body
        """
        
        payload = json.loads(paths)
        base_url = "http://localhost:3000/upload"

        result = requests.post(base_url, json=payload )
        
        return result.json()
    
    
    def _arun(self, paths) -> str:
        raise NotImplementedError("This tool does not support async")

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

api_key = os.getenv("OPENAI_API_KEY")
# initialize LLM (we use ChatOpenAI because we'll later define a `chat` agent)
llm = ChatOpenAI(
        openai_api_key=api_key,
        temperature=0,
        model_name='gpt-3.5-turbo'
)

# initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
        memory_key='chat_history',
        k=5,
        return_messages=True
)


In [ ]:
# from typing import Union

# class CircumferenceTool(BaseTool):
#     name = "Circumference calculator"
#     description = "use this tool when you need to calculate a circumference using the radius of a circle"

#     def _run(self, radius: Union[int, float]):
#         return float(radius)*2.0*pi

#     def _arun(self, radius: int):
#         raise NotImplementedError("This tool does not support async")


In [19]:
from langchain.agents import initialize_agent

tools = [IPFSUploadTool()]

# initialize agent with tools
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)


In [20]:
agent.run("""Can you uplad the files with this payload {
    "paths": [
        {
            "path": "paths/blob.json"
        }

    ]
} to IPFS please?""")



> Entering new  chain...
{
    "action": "IPFSUploadTool",
    "action_input": "{\"paths\": [{\"path\": \"paths/blob.json\"}]}"
}
Observation: bafybeif464sm7mbjkflhtwj2pjw7kii7oxm27euhu45a2rauy756zbs5ie
Thought:{
    "action": "Final Answer",
    "action_input": "bafybeif464sm7mbjkflhtwj2pjw7kii7oxm27euhu45a2rauy756zbs5ie"
}

> Finished chain.


'bafybeif464sm7mbjkflhtwj2pjw7kii7oxm27euhu45a2rauy756zbs5ie'

In [ ]:
https://ipfs.io/ipfs/bafybeif464sm7mbjkflhtwj2pjw7kii7oxm27euhu45a2rauy756zbs5ie